Transfer learning using cryptobert and roberta

In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, TextClassificationPipeline
from transformers import pipeline
import numpy as np
import pandas as pd
from scipy.special import softmax
import datetime

/Users/endy/.pyenv/versions/3.10.6/envs/crypto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install torch

In [3]:
!pip install protobuf==3.20.0

## Step 1 - Importing Bitcoin Tweet Data to begin analysing the model

In [4]:
# Kaggle - 2021Q1
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q1_reduced.csv')

# Kaggle - 2021Q2
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q2_reduced.csv')

# Kaggle - 2021Q3
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q2_reduced.csv')

# Kaggle - 2021Q4
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q2_reduced.csv')

In [40]:
# Kaggle - 202101, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202101_reduced.csv')
# Kaggle - 202102, issue
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202102_reduced.csv')
# data = data.drop([8120]) #error in row=8120
# Kaggle - 202103, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202103_reduced.csv')
# Kaggle - 202104, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202104_reduced.csv')
# Kaggle - 202105, issue 
data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202105_reduced.csv')
data = data.drop([6690]) #error in row=6690
# Kaggle - 202106, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202106_reduced.csv')
# Kaggle - 202107, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202107_reduced.csv')
# Kaggle - 202108, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202108_reduced.csv')
# Kaggle - 202109, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202109_reduced.csv')
# Kaggle - 202110, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202110_reduced.csv')
# Kaggle - 202111, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202111_reduced.csv')
# Kaggle - 202112, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202112_reduced.csv')

In [41]:
data

,Unnamed: 0,datetime,username,text,has_#bitcoin,date
0,61685,2021-05-01 22:09:58+00:00,Nihlahm14,Good project\n#punkprotocol #airdrop #giveaway...,True,2021-05-01
1,61686,2021-05-01 22:09:55+00:00,wearecryptoarmy,Demand for Cryptocoin Mixing Rises Significant...,True,2021-05-01
2,61687,2021-05-01 22:09:54+00:00,Devlishh,@moonxbtee you are seriously asking me which i...,True,2021-05-01
3,61688,2021-05-01 22:09:54+00:00,dogecrypty,"We rise again, stronger than ever...\n\n#dodge...",True,2021-05-01
4,61689,2021-05-01 22:09:46+00:00,Alaskacryptogi1,#Bitcoin this is absolutely irony. Every gover...,True,2021-05-01
...,...,...,...,...,...,...
20014,81699,2021-05-31 06:00:02+00:00,BitcoinBellyB,"The price of\nSpaghetti alla Carbonara\nis 31,...",True,2021-05-31
20015,81700,2021-05-31 06:00:01+00:00,takeoff_tech,"【Our Logo】\n\nEveryone’s dreams, all kinds of ...",True,2021-05-31
20016,81701,2021-05-31 06:00:01+00:00,PlatinumCrypto,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,True,2021-05-31
20017,81702,2021-05-31 06:00:01+00:00,lemalive,Monday morning.\nJust woke up to win a fucking...,True,2021-05-31


In [42]:
data = data[['text', 'date']]

In [43]:
data

,text,date
0,Good project\n#punkprotocol #airdrop #giveaway...,2021-05-01
1,Demand for Cryptocoin Mixing Rises Significant...,2021-05-01
2,@moonxbtee you are seriously asking me which i...,2021-05-01
3,"We rise again, stronger than ever...\n\n#dodge...",2021-05-01
4,#Bitcoin this is absolutely irony. Every gover...,2021-05-01
...,...,...
20014,"The price of\nSpaghetti alla Carbonara\nis 31,...",2021-05-31
20015,"【Our Logo】\n\nEveryone’s dreams, all kinds of ...",2021-05-31
20016,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,2021-05-31
20017,Monday morning.\nJust woke up to win a fucking...,2021-05-31


## Step 2 - Cleaning the data

In [44]:
# I have changed this to remove more information

def preprocess(text):
    new_text = []
    text = str(text)
    text = text.replace("\n", " ")
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [45]:
#Create a dummy data
data['process_text'] = data.text
# data['negative_bert'] = data.text
# data['neutral_bert'] = data.text
# data['positive_bert'] = data.text

#use the preprocess to clean the data
data['process_text'] = data['text'].apply(preprocess)

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10520/2241689511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['process_text'] = data.text
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10520/2241689511.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['process_text'] = data['text'].apply(preprocess)


In [46]:
data

,text,date,process_text
0,Good project\n#punkprotocol #airdrop #giveaway...,2021-05-01,Good project #punkprotocol #airdrop #giveaway ...
1,Demand for Cryptocoin Mixing Rises Significant...,2021-05-01,Demand for Cryptocoin Mixing Rises Significant...
2,@moonxbtee you are seriously asking me which i...,2021-05-01,you are seriously asking me which is my favou...
3,"We rise again, stronger than ever...\n\n#dodge...",2021-05-01,"We rise again, stronger than ever... #dodgeco..."
4,#Bitcoin this is absolutely irony. Every gover...,2021-05-01,#Bitcoin this is absolutely irony. Every gover...
...,...,...,...
20014,"The price of\nSpaghetti alla Carbonara\nis 31,...",2021-05-31,"The price of Spaghetti alla Carbonara is 31,87..."
20015,"【Our Logo】\n\nEveryone’s dreams, all kinds of ...",2021-05-31,"【Our Logo】 Everyone’s dreams, all kinds of dr..."
20016,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,2021-05-31,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...
20017,Monday morning.\nJust woke up to win a fucking...,2021-05-31,Monday morning. Just woke up to win a fucking ...


## Step 3 - analyse the language and sentiments by pretrained model

In [47]:
!pyenv local crypto

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [48]:
#Model 2b - Use the bert model, with full scores
MODEL_bert = f"ElKulako/cryptobert"
tokenizer_bert = AutoTokenizer.from_pretrained(MODEL_bert)
tokenizer_bert.model_max_length = 512 #solve the error: RuntimeError: The expanded size of the tensor (562) must match the existing size (514) at non-singleton dimension
config_bert = AutoConfig.from_pretrained(MODEL_bert)


# PT
model_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_bert)
model_bert.config.max_position_embeddings = 512

In [49]:
def scores_bert(sample_text):
    encoded_input_bert = tokenizer_bert(sample_text, return_tensors='pt')
    output_bert = model_bert(**encoded_input_bert)
    scores_bert = output_bert[0][0].detach().numpy()
    scores_bert = softmax(scores_bert) #1st score is negative, 2nd score is netural, 3rd score is positive
    return scores_bert

In [50]:
data['text'] = data['process_text'].apply(scores_bert)

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10520/2559237736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['process_text'].apply(scores_bert)


In [51]:
data = data[['date','process_text','text']]

In [52]:
data

,date,process_text,text
0,2021-05-01,Good project #punkprotocol #airdrop #giveaway ...,"[0.0003009937, 0.53378063, 0.4659184]"
1,2021-05-01,Demand for Cryptocoin Mixing Rises Significant...,"[0.0013588017, 0.5690318, 0.42960945]"
2,2021-05-01,you are seriously asking me which is my favou...,"[7.4208736e-05, 0.13473623, 0.86518955]"
3,2021-05-01,"We rise again, stronger than ever... #dodgeco...","[0.00022440775, 0.38404274, 0.61573285]"
4,2021-05-01,#Bitcoin this is absolutely irony. Every gover...,"[0.07309254, 0.49250275, 0.4344048]"
...,...,...,...
20014,2021-05-31,"The price of Spaghetti alla Carbonara is 31,87...","[0.023289697, 0.53225714, 0.44445318]"
20015,2021-05-31,"【Our Logo】 Everyone’s dreams, all kinds of dr...","[0.00023141247, 0.24250025, 0.7572683]"
20016,2021-05-31,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,"[0.00013844737, 0.6710398, 0.3288217]"
20017,2021-05-31,Monday morning. Just woke up to win a fucking ...,"[0.0015162149, 0.3077324, 0.69075143]"


In [53]:
data["negative_bert"] = data["text"].map(lambda x: x[0])
data["neutral_bert"] = data["text"].map(lambda x: x[1])
data["positive_bert"] = data["text"].map(lambda x: x[2])

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10520/2546257067.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["negative_bert"] = data["text"].map(lambda x: x[0])
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10520/2546257067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["neutral_bert"] = data["text"].map(lambda x: x[1])
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10520/2546257067.py:3: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [54]:
data

,date,process_text,text,negative_bert,neutral_bert,positive_bert
0,2021-05-01,Good project #punkprotocol #airdrop #giveaway ...,"[0.0003009937, 0.53378063, 0.4659184]",0.000301,0.533781,0.465918
1,2021-05-01,Demand for Cryptocoin Mixing Rises Significant...,"[0.0013588017, 0.5690318, 0.42960945]",0.001359,0.569032,0.429609
2,2021-05-01,you are seriously asking me which is my favou...,"[7.4208736e-05, 0.13473623, 0.86518955]",0.000074,0.134736,0.865190
3,2021-05-01,"We rise again, stronger than ever... #dodgeco...","[0.00022440775, 0.38404274, 0.61573285]",0.000224,0.384043,0.615733
4,2021-05-01,#Bitcoin this is absolutely irony. Every gover...,"[0.07309254, 0.49250275, 0.4344048]",0.073093,0.492503,0.434405
...,...,...,...,...,...,...
20014,2021-05-31,"The price of Spaghetti alla Carbonara is 31,87...","[0.023289697, 0.53225714, 0.44445318]",0.023290,0.532257,0.444453
20015,2021-05-31,"【Our Logo】 Everyone’s dreams, all kinds of dr...","[0.00023141247, 0.24250025, 0.7572683]",0.000231,0.242500,0.757268
20016,2021-05-31,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,"[0.00013844737, 0.6710398, 0.3288217]",0.000138,0.671040,0.328822
20017,2021-05-31,Monday morning. Just woke up to win a fucking ...,"[0.0015162149, 0.3077324, 0.69075143]",0.001516,0.307732,0.690751


In [55]:
data = data[['date', 'process_text', 'negative_bert', 'neutral_bert','positive_bert']]

## Step 4: Count the total number of labels/scores (positive, negative vs neutral) by date

In [56]:
grouped_data = pd.DataFrame(data.groupby(['date'])[['negative_bert', 'neutral_bert', 'positive_bert']].sum().fillna(0).reset_index())
grouped_data = grouped_data.rename_axis("", axis="columns")

In [57]:
grouped_data

,date,negative_bert,neutral_bert,positive_bert
0,2021-05-01,25.617947,196.129501,200.252548
1,2021-05-02,21.812752,197.257355,179.929901
2,2021-05-03,31.451441,233.620285,225.928284
3,2021-05-04,22.814571,247.993668,228.191772
4,2021-05-05,33.597691,251.846176,233.556122
5,2021-05-06,24.407339,228.841019,212.751648
6,2021-05-07,29.954519,254.069611,230.975861
7,2021-05-08,25.704597,211.071732,203.223663
8,2021-05-09,18.460251,173.351654,173.188095
9,2021-05-10,31.686495,235.573410,240.740097


## Step 5 - Download the data

In [58]:
start_date = grouped_data['date'][0]
# start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
start_date

'2021-05-01'

In [59]:
end_date = grouped_data['date'][len(grouped_data)-1]
# end_date_str = datetime.datetime.strftime(end_date, "%Y-%m-%d")
end_date

'2021-05-31'

In [60]:
file_name = f"{start_date}_{end_date}_twitter_comments.csv"
data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")

In [61]:
file_name = f"{start_date}_{end_date}_twitter_transferlearning.csv"
grouped_data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")